In [1]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

import tensorflow as tf
from SMDF.logging import logger

In [2]:
import joblib
import datetime

In [3]:
pwd

'd:\\Full Stack Data Science\\Time Series Analysis\\MAJOR PROJECT\\SMDF\\research'

In [4]:
cd ..

d:\Full Stack Data Science\Time Series Analysis\MAJOR PROJECT\SMDF


In [5]:
from pathlib import Path

In [6]:
class ForecastingPipeline:
    def __init__(self):
        self.M01AB = tf.saved_model.load(Path("artifacts/model_trainer/M01AB"))
        self.M01AE = tf.saved_model.load("artifacts/model_trainer/M01AE")
        self.N02BA = tf.saved_model.load("artifacts/model_trainer/N02BA")
        self.N02BE = tf.saved_model.load("artifacts/model_trainer/N02BE")
        self.N05B = tf.saved_model.load("artifacts/model_trainer/N05B")
        self.N05C = tf.saved_model.load("artifacts/model_trainer/N05C")
        self.R03 = tf.saved_model.load("artifacts/model_trainer/R03")
        self.R06 = tf.saved_model.load("artifacts/model_trainer/R06")
        self.train = pd.read_csv("artifacts/data_spliting/train.csv")
        self.test = pd.read_csv("artifacts/data_spliting/test.csv")
        self.scaler = joblib

    def noramalizing_data(self,column_name):
        scaler = StandardScaler()
        #self.train.drop(columns=["datum","Year","Month","Hour","Weekday Name"],inplace=True)
        #self.test.drop(columns=["datum","Year","Month","Hour","Weekday Name"],inplace=True)

        train_data = scaler.fit_transform(self.train[column_name].values.reshape(-1,1))

        test_data = scaler.transform(self.test[column_name].values.reshape(-1,1))

        
        logger.info(test_data.shape)
        
        #print(test_data)
        return  test_data,train_data,scaler
    
    def test_spliting(self,column_name):
        train, test,scaler = self.noramalizing_data(column_name)
        window_size = 60
        # Concatenate train data to test data
        dataset_total = np.concatenate((train, test), axis = 0)
        # Split test data and last window-size of train data
        inputs = dataset_total[len(dataset_total) - len(test) - window_size:]
        # Do the same thing for test data
        X_test = []
        y_test = []
        for i in range(window_size, window_size+len(test)):
            X_test.append(inputs[i-window_size:i,:]) 
            y_test.append(inputs[i,-1]) # consider Close as target
        # Change them to numpy array
        X_test, y_test = np.array(X_test).astype('float32'), np.array(y_test).reshape(-1, 1)
        logger.info(X_test.shape)
    
        return X_test,y_test,scaler


    def helper_forecast(self,column_name,key):
        X_test, _,scaler = self.test_spliting(column_name)
        
        logger.info("Forecasting Start ")
        logger.info(X_test.shape)
        models = [self.M01AB,
                  self.M01AE,
                  self.N02BA,
                  self.N02BE,
                  self.N05B,
                  self.N05C,
                  self.R03,
                  self.R06]
        time_distribution = ['time_distributed_1',
                             'time_distributed_3',
                             'time_distributed_5',
                             'time_distributed_7',
                             'time_distributed_9',
                             'time_distributed_11',
                             'time_distributed_13',
                             'time_distributed_15']
        
        x_input=X_test[-1:].reshape(1,-1)
        
        
        temp_input=list(x_input)
        temp_input=temp_input[0].tolist()
        logger.info(len(temp_input))
        
    
        lst_output = []
        n_step = 60
        i = 0
        while i<30:
        
            if len(temp_input) >60:
                x_input=np.array(temp_input[1:])
                x_input=x_input.reshape(1,-1)
                x_input = x_input.reshape((1, n_step, 1))
                #print(x_input)
                #forecast.append(x_input)
                #print(forecast)
                #logger.info("60 >")
                x_input = tf.constant(x_input, dtype=tf.float32)
                model = models[key].signatures["serving_default"]
                test_predict = model(x_input)
                #logger.info(test_predict)
                #print(test_predict)
                predictions = test_predict[time_distribution[key]].numpy()
                temp_input=temp_input[1:]
                lst_output.extend(predictions[0][0].tolist())
                i=i+1

            else:
                x_input = np.array(x_input)
                x_input = x_input.reshape((1, n_step,1))
                
                #x_input = x_input.astype(float) 
                model = models[key].signatures["serving_default"]
                test_predict = model(tf.constant(x_input))
                #logger.info(test_predict)
                #print(test_predict)
                predictions = test_predict[time_distribution[key]].numpy()
                #yhat = model.predict(x_input, verbose=0)
                temp_input.extend(predictions[0][0].tolist())
                lst_output.extend(predictions[0][0].tolist())
                i = i+1
                #logger.info(type(temp_input[0]))
                    
       

        
        return lst_output,scaler
    def forecasting(self):
        
        today = datetime.datetime.now().strftime('%m/%d/%Y')
        forecast_dates=pd.date_range(start=today, periods = 30)
        forecast=pd.DataFrame(forecast_dates,columns=['Date'])
        forecast['Date']=pd.to_datetime(forecast['Date'])
        forecast.sort_values('Date',inplace=True)
        columns = ["M01AB",
            "M01AE",
            "N02BA",
            "N02BE",
            "N05B",
            "N05C",
            "R03",
            "R06"]
        for i,val in enumerate(columns):
            forecast_data,scaler = self.helper_forecast(val,i)
            forecast_data=scaler.inverse_transform(np.array(forecast_data).reshape(-1, 1))
            forecast[val] = forecast_data
            
        return forecast.to_csv("artifacts/forecast_data/forecast.csv",index=False),forecast
    
                
        
    

In [7]:
forecast_cls = ForecastingPipeline()
forecast_cls.forecasting()


[2023-07-03 08:24:13,481: INFO: 3442374582: (422, 1)]
[2023-07-03 08:24:13,498: INFO: 3442374582: (1684, 60, 1)]
[2023-07-03 08:24:13,511: INFO: 3442374582: Forecasting Start ]
[2023-07-03 08:24:13,511: INFO: 3442374582: (1684, 60, 1)]
[2023-07-03 08:24:13,511: INFO: 3442374582: 60]
[2023-07-03 08:24:16,231: INFO: 3442374582: (422, 1)]
[2023-07-03 08:24:16,246: INFO: 3442374582: (1684, 60, 1)]
[2023-07-03 08:24:16,246: INFO: 3442374582: Forecasting Start ]
[2023-07-03 08:24:16,246: INFO: 3442374582: (1684, 60, 1)]
[2023-07-03 08:24:16,252: INFO: 3442374582: 60]
[2023-07-03 08:24:16,631: INFO: 3442374582: (422, 1)]
[2023-07-03 08:24:16,641: INFO: 3442374582: (1684, 60, 1)]
[2023-07-03 08:24:16,641: INFO: 3442374582: Forecasting Start ]
[2023-07-03 08:24:16,646: INFO: 3442374582: (1684, 60, 1)]
[2023-07-03 08:24:16,646: INFO: 3442374582: 60]
[2023-07-03 08:24:17,026: INFO: 3442374582: (422, 1)]
[2023-07-03 08:24:17,031: INFO: 3442374582: (1684, 60, 1)]
[2023-07-03 08:24:17,036: INFO: 344

(None,
          Date      M01AB     M01AE     N02BA      N02BE       N05B      N05C  \
 0  2023-07-03  10.862004  4.971833  6.038793  16.590708   6.143906 -0.060594   
 1  2023-07-04   8.090595  4.390884  4.598959  16.222011  16.128204  0.142665   
 2  2023-07-05   8.090595  4.390884  4.598959  16.222011  16.128204  0.142665   
 3  2023-07-06   5.574401  3.633947  3.364209  19.972835   6.323010  0.462129   
 4  2023-07-07   5.574401  3.633947  3.364209  19.972835   6.323010  0.462129   
 5  2023-07-08   0.811251  1.929617  3.016167  20.056146   8.635968  0.433374   
 6  2023-07-09   0.811251  1.929617  3.016167  20.056146   8.635968  0.433374   
 7  2023-07-10   7.820029  3.316140  1.492194  25.510817   6.302610  0.824410   
 8  2023-07-11   7.820029  3.316140  1.492194  25.510817   6.302610  0.824410   
 9  2023-07-12   4.491529  3.640577  2.418786  23.047793   6.924994  0.127611   
 10 2023-07-13   4.491529  3.640577  2.418786  23.047793   6.924994  0.127611   
 11 2023-07-14   3.44